# Keras - Mnist - image recognition

## imports

In [1]:
from __future__ import print_function
import numpy as np
import math
import keras
import cv2
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import RMSprop
from scipy import ndimage

Using TensorFlow backend.


In [2]:
batch_size=128
num_classes=10
epochs=20

In [3]:
# the data ,shuffled and split between train and test sets
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [4]:
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)

x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

x_train /=255
x_test /=255

In [5]:
print(x_train.shape[0],'train samples')
print(x_test.shape[0],'test samples')

60000 train samples
10000 test samples


In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
#convert class vectors to binary class matrices
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [8]:
model=Sequential()
model.add(Dense(512,activation='relu',input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes,activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 7s 110us/step - loss: 0.2434 - acc: 0.9244 - val_loss: 0.1019 - val_acc: 0.9671
Epoch 2/20
60000/60000 [==============================] - 6s 105us/step - loss: 0.1024 - acc: 0.9691 - val_loss: 0.0806 - val_acc: 0.9745
Epoch 3/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0747 - acc: 0.9767 - val_loss: 0.0746 - val_acc: 0.9785
Epoch 4/20
60000/60000 [==============================] - 6s 105us/step - loss: 0.0596 - acc: 0.9823 - val_loss: 0.0750 - val_acc: 0.9805
Epoch 5/20
60000/60000 [==============================] - 6s 105us/step - loss: 0.0504 - acc: 0.9849 - val_loss: 0.0733 - val_acc: 0.9803
Epoch 6/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0443 - acc: 0.9870 - val_loss: 0.0825 - val_acc: 0.9799
Epoch 7/20
60000/60000 [==============================] - 6s 106us/step - loss: 0.0396 - acc: 0.9880 - val_loss: 0.0849 - 

In [15]:
score = model.evaluate(x_test, y_test, verbose=0)

In [16]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0962843564058
Test accuracy: 0.9845


In [17]:
def getBestShift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

In [18]:
def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

In [26]:
def test_own_img(img):
    gray = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    gray = cv2.resize(255-gray, (28, 28))
    (thresh, gray) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    cv2.imwrite("not"+img, gray)
    while np.sum(gray[0]) == 0:
        gray = gray[1:]

    while np.sum(gray[:,0]) == 0:
        gray = np.delete(gray,0,1)

    while np.sum(gray[-1]) == 0:
        gray = gray[:-1]

    while np.sum(gray[:,-1]) == 0:
        gray = np.delete(gray,-1,1)

    rows,cols = gray.shape

    if rows > cols:
        factor = 20.0/rows
        rows = 20
        cols = int(round(cols*factor))
        # first cols than rows
        gray = cv2.resize(gray, (cols,rows))
    else:
        factor = 20.0/cols
        cols = 20
        rows = int(round(rows*factor))
        # first cols than rows
        gray = cv2.resize(gray, (cols, rows))
    
    
    colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
    rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
    
    gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')

    shiftx,shifty = getBestShift(gray)
    shifted = shift(gray,shiftx,shifty)
    gray = shifted
    cv2.imwrite("not_1"+img, gray)
    gray = gray/255
    print(np.argmax(model.predict(gray.reshape(1,784))))

In [27]:
test_own_img('0.png')
test_own_img('3.png')
test_own_img('4.png')
test_own_img('8.png')
test_own_img('shekar_4.jpg')

16 20
(6, 6) (4, 4)
0
20 16
(4, 4) (6, 6)
3
20 14
(4, 4) (7, 7)
4
20 15
(4, 4) (7, 6)
8
17 20
(6, 5) (4, 4)
4
